<a href="https://colab.research.google.com/github/souradip93/CS69002_9A_18CS60R07/blob/master/SciBERT_RelationExtraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pytorch-pretrained-bert
!pip install jsonlines

     |████████████████████████████████| 133kB 8.7MB/s 


In [0]:
import torch
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM
import logging
from torch.utils.data import TensorDataset, DataLoader, SequentialSampler
import jsonlines
import torch.nn.functional as F

logging.basicConfig(level=logging.INFO)

Get dataset

In [3]:
!wget https://raw.githubusercontent.com/allenai/scibert/master/data/text_classification/sciie-relation-extraction/train.txt

--2019-05-31 14:58:42--  https://raw.githubusercontent.com/allenai/scibert/master/data/text_classification/sciie-relation-extraction/train.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 823836 (805K) [text/plain]
Saving to: ‘train.txt’

train.txt           100%[===================>] 804.53K  --.-KB/s    in 0.02s   

2019-05-31 14:58:42 (32.5 MB/s) - ‘train.txt’ saved [823836/823836]



        # The convention in BERT is:
        # (a) For sequence pairs:
        #  tokens:   [CLS] is this jack ##son ##ville ? [SEP] no it is not . [SEP]
        #  type_ids:   0   0  0    0    0     0      0   0    1  1  1   1  1   1
        # (b) For single sequences:
        #  tokens:   [CLS] the dog is hairy . [SEP]
        #  type_ids:   0   0   0   0  0     0   0
        #
        # Where "type_ids" are used to indicate whether this is the first
        # sequence or the second sequence. The embedding vectors for `type=0` and
        # `type=1` were learned during pre-training and are added to the wordpiece
        # embedding vector (and position vector). This is not *strictly* necessary
        # since the [SEP] token unambigiously separates the sequences, but it makes
        # it easier for the model to learn the concept of sequences.
        #
        # For classification tasks, the first vector (corresponding to [CLS]) is
        # used as as the "sentence vector". Note that this only makes sense because
        # the entire model is fine-tuned.

BERT TOKENIZER

In [30]:
#tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
tokenizer = BertTokenizer.from_pretrained('scibert_scivocab_uncased/vocab.txt', do_lower_case=True)

INFO:pytorch_pretrained_bert.tokenization:loading vocabulary file scibert_scivocab_uncased/vocab.txt


In [0]:
def read_(file_path):
  with jsonlines.open(file_path) as f_in:
    text_id = 0
    train_data = []
    for json_object in f_in:
        train_data.append(convert_text_to_features(
            text=json_object.get('text'),
            seq_length=120,
            label=json_object.get('label'),
            text_unique_id=text_id
        ))
        text_id += 1
  return train_data

In [0]:
def convert_text_to_features(text, seq_length, label, text_unique_id):
    """Loads a data file into a list of `InputFeature`s."""

    feature = {}
    
    tokens_a = tokenizer.tokenize(text)

    if len(tokens_a) > seq_length - 2:
      tokens_a = tokens_a[0:(seq_length - 2)]

    tokens = []
    input_type_ids = []
    tokens.append("[CLS]")
    input_type_ids.append(0)
    for token in tokens_a:
      tokens.append(token)
      input_type_ids.append(0)
    tokens.append("[SEP]")
    input_type_ids.append(0)

    input_ids = tokenizer.convert_tokens_to_ids(tokens)
    input_mask = [1] * len(input_ids)

    # Zero-pad up to the sequence length.
    while len(input_ids) < seq_length:
      input_ids.append(0)
      input_mask.append(0)
      input_type_ids.append(0)

    assert len(input_ids) == seq_length
    assert len(input_mask) == seq_length
    assert len(input_type_ids) == seq_length

    feature['unique_id'] = text_unique_id
    feature['tokens'] = tokens
    feature['input_ids'] = input_ids
    feature['input_mask'] = input_mask
    feature['input_type_ids'] = input_type_ids
        
    return feature,label

In [0]:
train_data = read_('train.txt')

In [0]:
labels_list = []
for _,label in train_data:
  if label not in labels_list:
    labels_list.append(label)
labels_list

def get_label_index(label):
  return labels_list.index(label)

In [35]:
device = torch.device("cuda")
#model = BertModel.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('scibert_scivocab_uncased/weights.tar.gz')
model.to(device)

INFO:pytorch_pretrained_bert.modeling:loading archive file scibert_scivocab_uncased/weights.tar.gz
INFO:pytorch_pretrained_bert.modeling:extracting archive file scibert_scivocab_uncased/weights.tar.gz to temp dir /tmp/tmpfbobjgma
INFO:pytorch_pretrained_bert.modeling:Model config {
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "type_vocab_size": 2,
  "vocab_size": 31090
}



BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(31090, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): BertLayerNorm()
    (dropout): Dropout(p=0.1)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): BertLayerNorm()
            (dropout): Dropout(p=0.1)
          )
        )
        (intermediate): BertIntermediate(
          (dense): Linear(in_features=768, out_features=3072, bias=

In [36]:
all_input_ids = torch.tensor([feature['input_ids'] for feature,_ in train_data], dtype=torch.long).to(device)
all_input_mask = torch.tensor([feature['input_mask'] for feature,_ in train_data], dtype=torch.long).to(device)
all_input_type_ids = torch.tensor([feature['input_type_ids'] for feature,_ in train_data], dtype=torch.long).to(device)
all_labels = torch.tensor([get_label_index(label) for _,label in train_data], dtype=torch.long).to(device)
all_example_index = torch.arange(all_input_ids.size(0), dtype=torch.long).to(device)

all_input_ids.size()

torch.Size([3219, 120])

In [0]:
eval_data = TensorDataset(all_input_ids, all_input_mask, all_input_type_ids, all_labels, all_example_index)
eval_sampler = SequentialSampler(eval_data)
eval_dataloader = DataLoader(eval_data, sampler=eval_sampler, batch_size=16)

In [0]:
class BertEmbedder(torch.nn.Module):
  def __init__(self, bert_model):
    super(BertEmbedder, self).__init__()
    
    self.bert_model = bert_model
    self.output_dim = bert_model.config.hidden_size
    
  def forward(self, input_ids, input_mask, token_type_ids):
    all_encoder_layers,_ = self.bert_model(input_ids=input_ids, token_type_ids=token_type_ids, attention_mask=input_mask)
    all_encoder_layers = torch.stack(all_encoder_layers)
    embedding = all_encoder_layers[-1]
    return embedding
  
  def get_output_dim(self):
    return self.output_dim

In [0]:
class BertForClassification(torch.nn.Module):
  def __init__(self, text_embedder, dropout: float = 0.2):
        
    super(BertForClassification, self).__init__()
    self.text_embedder = text_embedder
    self.dropout = torch.nn.Dropout(dropout)
    self.num_classes = len(labels_list)
    self.classifier_feedforward = torch.nn.Linear(self.text_embedder.get_output_dim()  , self.num_classes).to(device)
    self.loss = torch.nn.CrossEntropyLoss()
               
               
  def forward(self, input_ids, input_mask, input_type_ids):
    embedded_text = self.text_embedder(input_ids, input_mask, input_type_ids)
    pooled = self.dropout(embedded_text[:, 0, :])
    logits = self.classifier_feedforward(pooled)
    #class_probs = F.softmax(logits, dim=1)
    
    #output_dict = {"logits": logits}
    #output_dict["class_probs"] = class_probs
    #loss = self.loss(class_probs, label)
    #output_dict["loss"] = loss
    #return output_dict
    return logits

In [18]:
bert_embedder = BertEmbedder(model)

for input_ids, input_mask, input_type_ids, labels, example_indices in eval_dataloader:
  embedding = bert_embedder(input_ids, input_mask, input_type_ids)
  #print(embedding.size())
  
embedding[-1].size()

torch.Size([120, 768])

In [0]:
print(torch.cuda.get_device_properties(device).total_memory)

15812263936


In [42]:
#bert_embedder = BertEmbedder(model)
#bert_classifier = BertForClassification(bert_embedder)

#param_optimizer = list(model.named_parameters())
#no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
#optimizer_grouped_parameters = [
#    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
#    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
#    ]
#optimizer = torch.optim.Adam(optimizer_grouped_parameters, lr=0.0001)
#optimizer.zero_grad()

for eopch in range(15):
  tr_loss = 0
  correct_count,total_count = 0,0
  for input_ids, input_mask, input_type_ids, labels, example_indices in eval_dataloader:
    logits = bert_classifier(input_ids, input_mask, input_type_ids)
    #loss = output_dict['loss']
    loss = bert_classifier.loss(logits, labels)

    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

    tr_loss += loss.item()
    
    #num_classes = output_dict['class_probs'].size(-1)
    #predictions = output_dict['class_probs'].view((-1, num_classes))
    
    num_classes = logits.size(-1)
    predictions = logits.view((-1, num_classes))
    gold_labels = labels.view(-1).long()
    
    predictions = predictions.view((-1, num_classes))
    gold_labels = gold_labels.view(-1).long()
    
    top_k = predictions.max(-1)[1].unsqueeze(-1)
    correct = top_k.eq(gold_labels.unsqueeze(-1)).float()
    correct_count += correct.sum()
    total_count += gold_labels.numel()
    
  accuracy = float(correct_count) / float(total_count)

  print(tr_loss,accuracy)


40.86948500573635 0.932587760173967
34.73544389009476 0.9425287356321839
35.758873203769326 0.9415967691829761
33.069150384515524 0.949363156259708
28.060105085372925 0.95278036657347
26.336552642285824 0.9577508543025784
25.135287575423717 0.9589934762348555
24.858046356588602 0.9648959304131718
40.59861145541072 0.9338303821062441
37.712253373116255 0.9369369369369369
34.879836241714656 0.9465672569120845
28.294928221032023 0.9540229885057471
22.955023393034935 0.961789375582479
26.70872782636434 0.9596147872009941
24.42994167841971 0.958372165268717


In [43]:
!wget https://raw.githubusercontent.com/allenai/scibert/master/data/text_classification/sciie-relation-extraction/test.txt

--2019-05-31 17:56:26--  https://raw.githubusercontent.com/allenai/scibert/master/data/text_classification/sciie-relation-extraction/test.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 249465 (244K) [text/plain]
Saving to: ‘test.txt.1’

test.txt.1          100%[===================>] 243.62K  --.-KB/s    in 0.02s   

2019-05-31 17:56:27 (14.4 MB/s) - ‘test.txt.1’ saved [249465/249465]



In [0]:
test_data = read_('test.txt')

In [0]:
labels_list = []
for _,label in test_data:
  if label not in labels_list:
    labels_list.append(label)
labels_list

def get_label_index(label):
  return labels_list.index(label)

In [46]:
all_input_ids = torch.tensor([feature['input_ids'] for feature,_ in test_data], dtype=torch.long).to(device)
all_input_mask = torch.tensor([feature['input_mask'] for feature,_ in test_data], dtype=torch.long).to(device)
all_input_type_ids = torch.tensor([feature['input_type_ids'] for feature,_ in test_data], dtype=torch.long).to(device)
all_labels = torch.tensor([get_label_index(label) for _,label in test_data], dtype=torch.long).to(device)
all_example_index = torch.arange(all_input_ids.size(0), dtype=torch.long).to(device)

all_input_ids.size()

torch.Size([974, 120])

In [0]:
eval_data = TensorDataset(all_input_ids, all_input_mask, all_input_type_ids, all_labels, all_example_index)
eval_sampler = SequentialSampler(eval_data)
eval_dataloader = DataLoader(eval_data, sampler=eval_sampler, batch_size=16)

In [48]:
for input_ids, input_mask, input_type_ids, labels, example_indices in eval_dataloader:
  logits = bert_classifier(input_ids, input_mask, input_type_ids)
  class_probs = F.softmax(logits, dim=1)

  num_classes = class_probs.size(-1)
  predictions = class_probs.view((-1, num_classes))
  gold_labels = labels.view(-1).long()

  predictions = predictions.view((-1, num_classes))
  gold_labels = gold_labels.view(-1).long()

  top_k = predictions.max(-1)[1].unsqueeze(-1)
  correct = top_k.eq(gold_labels.unsqueeze(-1)).float()
  correct_count += correct.sum()
  total_count += gold_labels.numel()
    
accuracy = float(correct_count) / float(total_count)
print(accuracy)

0.7693775339852135


In [24]:
!wget https://s3-us-west-2.amazonaws.com/ai2-s2-research/scibert/pytorch_models/scibert_scivocab_uncased.tar

--2019-05-31 16:43:38--  https://s3-us-west-2.amazonaws.com/ai2-s2-research/scibert/pytorch_models/scibert_scivocab_uncased.tar
Resolving s3-us-west-2.amazonaws.com (s3-us-west-2.amazonaws.com)... 52.218.217.136
Connecting to s3-us-west-2.amazonaws.com (s3-us-west-2.amazonaws.com)|52.218.217.136|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 410593280 (392M) [application/x-tar]
Saving to: ‘scibert_scivocab_uncased.tar’

scibert_scivocab_un 100%[===================>] 391.57M  20.2MB/s    in 20s     

2019-05-31 16:43:59 (19.6 MB/s) - ‘scibert_scivocab_uncased.tar’ saved [410593280/410593280]



In [25]:
!tar -xvf scibert_scivocab_uncased.tar

scibert_scivocab_uncased/
scibert_scivocab_uncased/weights.tar.gz
scibert_scivocab_uncased/vocab.txt


In [27]:
bert_tokenizer = BertTokenizer.from_pretrained('scibert_scivocab_uncased/vocab.txt', do_lower_case=True)

INFO:pytorch_pretrained_bert.tokenization:loading vocabulary file scibert_scivocab_uncased/vocab.txt
